In [47]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, plot_confusion_matrix
from sklearn.metrics import auc,roc_curve,roc_auc_score

import tensorflow as tf
print("Tensorflow version:", tf.__version__)

import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, array_to_img, img_to_array
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

from IPython.display import SVG, Image

!pip install livelossplot
from livelossplot.tf_keras import PlotLossesCallback

import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

## Exploratory Data Analysis

In [48]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename));

In [49]:
#define path to the data directory
base_dir = '/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/'

In [50]:
os.listdir(base_dir)

In [51]:
#read the entire dataset
df = pd.read_csv(base_dir+'icml_face_data.csv')
df.columns = ['emotion', 'Usage', 'pixels']
df.head()

In [52]:
df.info()

In [53]:
df['emotion'].value_counts()

In [54]:
df['Usage'].value_counts()

In [55]:
#read train data
train = pd.read_csv(base_dir+'train.csv')
train.head()

In [57]:
#read test data
test = pd.read_csv(base_dir+'test.csv')
test.head()

In [59]:
emotions = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

In [60]:
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(30,8))

sns.countplot(data = df[df['Usage']=='Training'], x='emotion', ax=ax1, palette='Spectral').set_title('Training')
ax1.set_xticklabels(emotions.values())

sns.countplot(data = df[df['Usage']=='PublicTest'], x='emotion', ax=ax2, palette='Spectral').set_title('Testing')
ax2.set_xticklabels(emotions.values())

sns.countplot(data = df[df['Usage']=='PrivateTest'], x='emotion', ax=ax3, palette='Spectral').set_title('Validation')
ax3.set_xticklabels(emotions.values())

In [61]:
fig = plt.figure(1, (20, 20))

k = 0
for label in sorted(df['emotion'].unique()):
    for j in range(7):
        px = df[df['emotion']==label].pixels.iloc[k]
        px = np.array(px.split(' ')).reshape(48, 48).astype('float32')

        k += 1
        ax = plt.subplot(7, 7, k)
        ax.imshow(px, cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(emotions[label])
        plt.tight_layout()

## Prepare Train & Test Set Inputs
### Train Set

In [62]:
train_data = df[df['Usage']=='Training']
train_data.drop(columns='Usage', inplace=True)
train_data.head()

In [63]:
train_data.isnull().sum()

In [64]:
train_data.info()

In [65]:
train_data['pixels'][0]

In [66]:
def prepare_data(data):
    image_array = np.zeros(shape=(len(data), 48, 48, 1))
    image_label = np.array(list(map(int, data['emotion'])))

    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, 'pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48)) 
        image_array[i, :, :, 0] = image / 255

    return image_array, image_label

In [67]:
X_train, y_train = prepare_data(train_data)
print(X_train.shape, y_train.shape)

In [68]:
X_train

In [69]:
y_train

### Test Set

In [70]:
test_data = df[df['Usage']!='Training']
test_data.drop(columns='Usage', inplace=True)
test_data.head()

In [71]:
X_test, y_test = prepare_data(test_data)
print(X_test.shape, y_test.shape)

### Train Test Split

In [72]:
#split train set into training and validation set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                  test_size=0.2, 
                                                  random_state=121,
                                                  shuffle=True)

In [73]:
print('X_train', X_train.shape)
print('X_test', X_test.shape)
print('X_val', X_val.shape)

print('y_train', y_train.shape)
print('y_test', y_test.shape)
print('y_val', y_val.shape)

### Prepare Labels

In [74]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)
y_val = keras.utils.to_categorical(y_val)

print('y_train', y_train.shape)
print('y_test', y_test.shape)
print('y_val', y_val.shape)

In [75]:
y_train

In [76]:
def plot_examples(label):
    fig, axs = plt.subplots(1, 5, figsize=(20, 8))
    fig.subplots_adjust(hspace = .2, wspace=.2)
    axs = axs.ravel()
    for i in range(5):
        idx = train_data[train_data['emotion']==label].index[i]
        axs[i].imshow(X_train[idx][:,:,0], cmap='gray')
        axs[i].set_title(emotions[label])
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])

In [77]:
plot_examples(label=0)

In [78]:
plot_examples(label=1)

In [79]:
plot_examples(label=2)

In [80]:
plot_examples(label=3)

In [81]:
plot_examples(label=4)

In [82]:
plot_examples(label=5)

In [83]:
plot_examples(label=6)

## Data Loader and Data Augmentation

In [84]:
img_size = 48
batch_size = 32

#use Image Data Generator to perform this task
#train set
datagen = ImageDataGenerator(rotation_range=25, 
                             width_shift_range=0.1,
                             height_shift_range=0.1, 
                             shear_range=0.2, 
                             zoom_range=0.2,
                             horizontal_flip=True)

datagen.fit(X_train)
datagen.fit(X_val)

## Build Model

In [85]:
model = Sequential()

#1st conv
model.add(Conv2D(64, (3,3), padding='same', input_shape=(48,48,1))) #1 is for grayscale
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#2nd conv
model.add(Conv2D(128, (5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#3rd conv
model.add(Conv2D(512, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#4th conv
model.add(Conv2D(512, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

In [86]:
#compile
model.compile(optimizer=keras.optimizers.Adam(learning_rate = 1e-5),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [87]:
model.summary()

In [88]:
from tensorflow.keras.utils import plot_model

#plot
plot_model(model, to_file = 'model_plot.png', show_shapes = True, show_layer_names = True)

## Train Model

In [89]:
epochs = 10
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_val)//batch_size

checkpoint = ModelCheckpoint('model_weights.h5',
                             monitor='val_accuracy',
                             save_weights_only=True,
                             mode='max',
                             verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.1,
                              patience=2,
                              min_lr=0.00001,
                              mode='auto')

earlystop = EarlyStopping(monitor='val_accuracy',
                          patience=5,
                          mode='auto',
                          verbose=1)

callbacks = [PlotLossesCallback(), earlystop, checkpoint, reduce_lr]

In [90]:
history = model.fit(X_train, y_train,
                   steps_per_epoch=steps_per_epoch,
                   epochs=epochs,
                   validation_data=(X_val, y_val),
                   validation_steps=validation_steps,
                   callbacks=callbacks)

## Evaluate Model 

In [91]:
fig , ax = plt.subplots(1,2)
fig.set_size_inches(20, 8)

train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

epochs = range(1, len(train_acc) + 1)

ax[0].plot(epochs , train_acc , 'g-o' , label = 'Training Accuracy')
ax[0].plot(epochs , val_acc , 'y-o' , label = 'Validation Accuracy')
ax[0].set_title('Model Training & Validation Accuracy')
ax[0].legend(loc = 'lower right')
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")

ax[1].plot(epochs , train_loss , 'g-o' , label = 'Training Loss')
ax[1].plot(epochs , val_loss , 'y-o' , label = 'Validation Loss')
ax[1].set_title('Model Training & Validation & Loss')
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")

plt.show()

In [92]:
print('Train accuracy & loss:', model.evaluate(X_train, y_train))
print('\n')
print('Test accuracy & loss:', model.evaluate(X_test, y_test))

In [93]:
#make prediction
yhat_test = np.argmax(model.predict(X_test), axis=1)
yhat_test

In [94]:
y_test = np.argmax(y_test, axis=1)
y_test

In [95]:
from mlxtend.plotting import plot_confusion_matrix

#get confusion matrix
cm = confusion_matrix(y_test, yhat_test)
print(cm)

fig, ax = plot_confusion_matrix(conf_mat=cm,
                                show_normed=True,
                                show_absolute=False,
                                class_names=emotions.values(),
                                figsize=(8, 8))
fig.show()

In [96]:
#get classification report
print(classification_report(y_test, yhat_test, target_names=emotions.values()))

In [97]:
model_json = model.to_json()
with open('model.json','w') as json_file:
    json_file.write(model_json)
    
model.save('final_model.h5')

## Make Prediction

In [ ]:
emotions_mapper = {0: 'angry',
                   1: 'disgust',
                   2: 'fear',
                   3: 'happy',
                   4: 'sad',
                   5: 'surprise',
                   6: 'neutral'}

In [103]:
random_angry_imgs = np.random.choice(np.where(y_test==0))

#angry
plt.subplot(1, 5, i+1)
sample_img = X_test[angryidx,:,:,0]
plt.imshow(sample_img, cmap='gray')
plt.set_xticks([])
plt.set_yticks([])
plt.set_title(f"true:angry, pred:{emotions_mapper[model.predict_classes(sample_img.reshape(1,48,48,1))[0]]}")

In [98]:
np.random.seed(21)
random_angry_imgs = np.random.choice(np.where(y_test[:, 1]==0)[0], size=5)
random_disgust_imgs = np.random.choice(np.where(y_test[:, 1]==1)[0], size=5)
random_fear_imgs = np.random.choice(np.where(y_test[:, 1]==2)[0], size=5)
random_happy_imgs = np.random.choice(np.where(y_test[:, 1]==3)[0], size=5)
random_sad_imgs = np.random.choice(np.where(y_test[:, 1]==4)[0], size=5)
random_surprise_imgs = np.random.choice(np.where(y_test[:, 5]==1)[0], size=5)
random_neutral_imgs = np.random.choice(np.where(y_test[:, 6]==1)[0], size=5)

fig = plt.figure(figsize=(30, 30))

for i, (angryidx, disgustidx, fearidx, 
        happyidx, sadidx, surpriseidx, neuidx) in enumerate(zip(random_angry_imgs, random_disgust_imgs, random_fear_imgs,
                                                                random_happy_imgs, random_sad_imgs, random_surprise_imgs, 
                                                                random_neutral_imgs)):
        #angry
        ax1 = plt.subplot(1, 5, i+1)
        sample_img = X_test[angryidx,:,:,0]
        ax1.imshow(sample_img, cmap='gray')
        ax1.set_xticks([])
        ax1.set_yticks([])
        ax1.set_title(f"true:angry, pred:{emotions_mapper[model.predict_classes(sample_img.reshape(1,48,48,1))[0]]}")

        #disgust
        ax2 = plt.subplot(1, 5, i+1)
        sample_img = X_test[disgustidx,:,:,0]
        ax2.imshow(sample_img, cmap='gray')
        ax2.set_xticks([])
        ax2.set_yticks([])
        ax2.set_title(f"true:disgust, pred:{emotions_mapper[model.predict_classes(sample_img.reshape(1,48,48,1))[0]]}")
        
        #fear
        ax3 = plt.subplot(1, 5, i+1)
        sample_img = X_test[fearidx,:,:,0]
        ax3.imshow(sample_img, cmap='gray')
        ax3.set_xticks([])
        ax3.set_yticks([])
        ax3.set_title(f"true:fear, pred:{emotions_mapper[model.predict_classes(sample_img.reshape(1,48,48,1))[0]]}")        
        
        #happy
        ax4 = plt.subplot(1, 5, i+1)
        sample_img = X_test[happyidx,:,:,0]
        ax4.imshow(sample_img, cmap='gray')
        ax4.set_xticks([])
        ax4.set_yticks([])
        ax4.set_title(f"true:happy, pred:{emotions_mapper[model.predict_classes(sample_img.reshape(1,48,48,1))[0]]}")        
        
        #sad
        ax5 = plt.subplot(1, 5, i+1)
        sample_img = X_test[sadidx,:,:,0]
        ax5.imshow(sample_img, cmap='gray')
        ax5.set_xticks([])
        ax5.set_yticks([])
        ax5.set_title(f"true:sad, pred:{emotions_mapper[model.predict_classes(sample_img.reshape(1,48,48,1))[0]]}")        
        
        #surprise
        ax6 = plt.subplot(1, 5, i+1)
        sample_img = X_test[surpriseidx,:,:,0]
        ax6.imshow(sample_img, cmap='gray')
        ax6.set_xticks([])
        ax6.set_yticks([])
        ax6.set_title(f"true:surpise, pred:{emotions_mapper[model.predict_classes(sample_img.reshape(1,48,48,1))[0]]}")        
        
        #neutral
        ax7 = pyplot.subplot(1, 5, i+1)
        sample_img = X_test[neuidx,:,:,0]
        ax7.imshow(sample_img, cmap='gray')
        ax7.set_xticks([])
        ax7.set_yticks([])
        ax7.set_title(f"t:neut, p:{emotions_mapper[model.predict_classes(sample_img.reshape(1,48,48,1))[0]]}")

        plt.tight_layout()

## FER

In [ ]:
!pip install fer

In [ ]:
fig = plt.figure(figsize=(20, 20))

for i in range(25):
    ax = fig.add_subplot(1, 5, i + 1)    
    image = np.random.choice(np.where(y_test[:, 1]==0)[0], size=5)
    detector = FER()
    detector.detect_emotions(img)

In [ ]:
from fer import FER
import cv2

img = cv2.imread("justin.jpg")
detector = FER()
detector.detect_emotions(img)

## Grad-CAM

In [ ]:
#https://github.com/gkeechin/vizgradcam/blob/main/gradcam.py

def VizGradCAM(model, image, interpolant=0.5, plot_results=True):

    """VizGradCAM - Displays GradCAM based on Keras / TensorFlow models
    using the gradients from the last convolutional layer. This function
    should work with all Keras Application listed here:
    https://keras.io/api/applications/
    Parameters:
    model (keras.model): Compiled Model with Weights Loaded
    image: Image to Perform Inference On
    plot_results (boolean): True - Function Plots using PLT
                            False - Returns Heatmap Array
    Returns:
    Heatmap Array?
    """
    #sanity check
    assert (interpolant > 0 and interpolant < 1), "Heatmap Interpolation Must Be Between 0 - 1"

    #STEP 1: Preprocesss image and make prediction using our model
    #input image
    original_img = np.asarray(image, dtype = np.float32)
    #expamd dimension and get batch size
    img = np.expand_dims(original_img, axis=0)
    #predict
    prediction = model.predict(img)
    #prediction index
    prediction_idx = np.argmax(prediction)

    #STEP 2: Create new model
    #specify last convolutional layer
    last_conv_layer = next(x for x in model.layers[::-1] if isinstance(x, K.layers.Conv2D))
    target_layer = model.get_layer(last_conv_layer.name)

    #compute gradient of top predicted class
    with tf.GradientTape() as tape:
        #create a model with original model inputs and the last conv_layer as the output
        gradient_model = Model([model.inputs], [target_layer.output, model.output])
        #pass the image through the base model and get the feature map  
        conv2d_out, prediction = gradient_model(img)
        #prediction loss
        loss = prediction[:, prediction_idx]

    #gradient() computes the gradient using operations recorded in context of this tape
    gradients = tape.gradient(loss, conv2d_out)

    #obtain the output from shape [1 x H x W x CHANNEL] -> [H x W x CHANNEL]
    output = conv2d_out[0]

    #obtain depthwise mean
    weights = tf.reduce_mean(gradients[0], axis=(0, 1))


    #create a 7x7 map for aggregation
    activation_map = np.zeros(output.shape[0:2], dtype=np.float32)
    #multiply weight for every layer
    for idx, weight in enumerate(weights):
        activation_map += weight * output[:, :, idx]
    #resize to image size
    activation_map = cv2.resize(activation_map.numpy(), 
                                (original_img.shape[1], 
                                 original_img.shape[0]))
    #ensure no negative number
    activation_map = np.maximum(activation_map, 0)
    #convert class activation map to 0 - 255
    activation_map = (activation_map - activation_map.min()) / (activation_map.max() - activation_map.min())
    #rescale and convert the type to int
    activation_map = np.uint8(255 * activation_map)


    #convert to heatmap
    heatmap = cv2.applyColorMap(activation_map, cv2.COLORMAP_JET)

    #superimpose heatmap onto image
    original_img = np.uint8((original_img - original_img.min()) / (original_img.max() - original_img.min()) * 255)
    cvt_heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    cvt_heatmap = img_to_array(cvt_heatmap)

    #enlarge plot
    plt.rcParams["figure.dpi"] = 100

    if plot_results == True:
        plt.imshow(np.uint8(original_img * interpolant + cvt_heatmap * (1 - interpolant)))
    else:
        return cvt_heatmap

In [ ]:
#load image
test_img = cv2.imread("/train/4.jpg")

#apply function
VizGradCAM(model, img_to_array(test_img), plot_results=True)

## Facial Recognition

In [ ]:
import dlib

frontalface_detector = dlib.get_frontal_face_detector()
def rect_to_bb(rect):
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
    return (x, y, w, h)
    
def detect_face(image_url):
    try:
        url_response = urllib.request.urlopen(image_url)
        img_array = np.array(bytearray(url_response.read()), dtype=np.uint8)
        image = cv2.imdecode(img_array, -1)
rects = frontalface_detector(image, 1)
if len(rects) < 1:
    return "No Face Detected"
for (i, rect) in enumerate(rects):
    (x, y, w, h) = rect_to_bb(rect)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
plt.imshow(image, interpolation='nearest')
plt.axis('off')
plt.show()

In [ ]:
frontalface_detector = dlib.get_frontal_face_detector()
landmark_predictor=dlib.shape_predictor('./shape_predictor_68_face_landmarks.dat')
def get_landmarks(image_url):
    try:
        url_response = urllib.request.urlopen(image_url)
        img_array = np.array(bytearray(url_response.read()), dtype=np.uint8)
        image = cv2.imdecode(img_array, -1)
    except Exception as e:
        print ("Please check the URL and try again!")
        return None,None
    faces = frontalface_detector(image, 1)
    if len(faces):
        landmarks = [(p.x, p.y) for p in landmark_predictor(image, faces[0]).parts()]
    else:
        return None,None
    
    return image,landmarks
def image_landmarks(image,face_landmarks):
    radius = -1
    circle_thickness = 4
    image_copy = image.copy()
    for (x, y) in face_landmarks:
        cv2.circle(image_copy, (x, y), circle_thickness, (255,0,0), radius)
        plt.imshow(image_copy, interpolation='nearest')
        plt.axis('off')
        plt.show()

## Represent Model as JSON String

In [ ]:
model_json = model.to_json()

with open('model.json', 'w') as json_file:
    json_file.write(model_json)